# Web Scraping Workshop 2 - HTML Parising with BeautifulSoup
Prepared by: Nickolas K. Freeman, Ph.D.

In this notebook, we will see how we can use Python to harvest data from HTML. In particular, we will be using the python `requests` and `beautifulsoup` libraries to request and parse financial data from Yahoo finance. 

The following code block imports some libraries that we will use.

In [12]:
import bs4
import datetime
import pandas as pd
import requests
import time

## Yahoo Finance

We will be harvesting data from `https://finance.yahoo.com`. Before starting, let's first look at the `robots.txt` file published for the site to see if there is any activity that is specifically *disallowed* or if any additional guidelines for access are given. The following code block makes a **GET** request for the site's `robots.txt` file and prints the returned text.

In [13]:
my_user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'

my_headers = {'User-Agent': my_user_agent, 
             'Referer': 'https://www.google.com'}

r = requests.get('https://finance.yahoo.com/robots.txt', headers = my_headers)
print(r.text)

User-agent: *
Sitemap: https://finance.yahoo.com/sitemap_en-us_desktop_index.xml
Sitemap: https://finance.yahoo.com/sitemaps/finance-sitemap_index_US_en-US.xml.gz
Sitemap: https://finance.yahoo.com/sitemaps/finance-sitemap_googlenewsindex_US_en-US.xml.gz
Disallow: /r/
Disallow: /_finance_doubledown/
Disallow: /nel_ms/
Disallow: /caas/
Disallow: /__rapidworker-1.2.js
Disallow: /__blank
Disallow: /_td_api
Disallow: /_remote
User-agent:googlebot
Disallow: /m/

User-agent:googlebot-news
Disallow: /m/



As you can see the `robots.txt` file specifies several paths that are disallowed for all user-agents. This is indicated by the `User-agent: *` line. However, also note that the files provides several sitemaps that actually help programs determine the layout of the site. We will use one of these sitemaps to help us access stock information for companies operating in specific sectors. Before doing this, let's store a list of prohibited paths so that we can ensure we do not volate the `robots.txt` directives.

In [14]:
prohibited_paths = []
for statement in r.text.split('\n'):
    if 'Disallow:' in statement:
        current_path = statement.strip('Disallow: ')
        prohibited_paths.append('https://finance.yahoo.com' + current_path)
prohibited_paths

['https://finance.yahoo.com/r/',
 'https://finance.yahoo.com/_finance_doubledown/',
 'https://finance.yahoo.com/nel_ms/',
 'https://finance.yahoo.com/caas/',
 'https://finance.yahoo.com/__rapidworker-1.2.j',
 'https://finance.yahoo.com/__blank',
 'https://finance.yahoo.com/_td_ap',
 'https://finance.yahoo.com/_remote',
 'https://finance.yahoo.com/m/',
 'https://finance.yahoo.com/m/']

The following code block makes a **GET** request for the first sitemap. The response from this request is stored in the variable `r`. We use the `content` attribute of this response to get a *bytes* representation of the reponse and use the *BeautifulSoup* library to convert this data into a `soup` object. We print a *prettified* version of the first 1,000 characters of this object. 

In [15]:
r = requests.get('https://finance.yahoo.com/sitemap_en-us_desktop_index.xml', headers = my_headers)

soup = bs4.BeautifulSoup(r.content)

print(soup.prettify()[:1000])

<?xml version="1.0"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/andy-serwer/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/brittany-jones-cooper/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/daniel-howley-20160419/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/daniel-roberts/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/julia-la-roche/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/mandi-woodruff/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/melody-hahm-20151026/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/nicole-sinclair/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/author/rick-newman/
  </loc>
 </url>
 <url>
  <loc>
   https://finance.yahoo.com/blogs/off-the-cuff/
  </loc>
 </url>
 <url>
  <

If we were to print more characters, you would see that there are URLS that follow the form ` https://finance.yahoo.com/sector/...`. These pages report stock information for top companies operating in the associated sector. The following code block shows how we can use BeautifulSoup to extract all of the `loc` elements that include the word `sector` in the associated text and store them in a list called `sector_urls`. Note how we are insuring that the returned URLS do not match any of the partial strings stored in our list of `prohibited_paths`.

In [16]:
sector_urls = []
for loc_tag in soup.find_all('loc'):
    if 'sector' in loc_tag.text:
        prohibited = False
        for path in prohibited_paths:
            if path in loc_tag.text:
                print('Path prohibited!')
                prohibited = True
        if not prohibited:
            print(f'{loc_tag.text} not prohibited')
            sector_urls.append(loc_tag.text)

https://finance.yahoo.com/sector/ms_basic_materials not prohibited
https://finance.yahoo.com/sector/ms_communication_services not prohibited
https://finance.yahoo.com/sector/ms_consumer_cyclical not prohibited
https://finance.yahoo.com/sector/ms_consumer_defensive not prohibited
https://finance.yahoo.com/sector/ms_energy not prohibited
https://finance.yahoo.com/sector/ms_financial_services not prohibited
https://finance.yahoo.com/sector/ms_healthcare not prohibited
https://finance.yahoo.com/sector/ms_industrials not prohibited
https://finance.yahoo.com/sector/ms_real_estate not prohibited
https://finance.yahoo.com/sector/ms_technology not prohibited
https://finance.yahoo.com/sector/ms_utilities not prohibited


The following code block prints the first URl in our `sector_urls` list. If you visit the site with your web browser, you sill see that the page displays a table of related stocks and associated information.

In [17]:
current_sector_loc = sector_urls[0]
print(sector_urls[0])

https://finance.yahoo.com/sector/ms_basic_materials


Inspecting the HTML of the previous URL, you will see that the table is enclosed in a set of `<table>` tags. The following code block 
1. requests the HTML associated with the URL, 
2. converts the response to a `soup` object, 
3. identifies all elements enclosed in `<table>` tags and stores the associated HTML in a list named `tables`, 
4. prints the number of items in the `tables` object, and
5. prints the first 5,000 characters of the first item in the `tables` list.

Note how the HTML for the tables uses the `<tr>`, `<th>`, and `<td>` to achieve the desired structure.

In [18]:
current_sector_loc = sector_urls[0]

r = requests.get(current_sector_loc)
soup = bs4.BeautifulSoup(r.content)

tables = soup.find_all('table')

print(f'The tables object has {len(tables)} item(s).\n')

print(f'The first item in the tables object is:\n{"-"*70}\n{tables[0].prettify()[:5000]}')

The tables object has 1 item(s).

The first item in the tables object is:
----------------------------------------------------------------------
<table class="W(100%)">
 <thead>
  <tr class="C($tertiaryColor) BdB Bdbc($seperatorColor)">
   <th class="Ta(start) Pstart(6px) Pend(10px) Miw(90px) Bgc($lv3BgColor) Fz(xs) Va(m) Py(5px)! Cur(p) Bgc($hoverBgColor):h Fw(400)! Ta(start) Start(0) Pend(10px) Pos(st) Bgc($lv3BgColor) Z(1) Ta(start)!">
    <label class="Ta(c) Pos(r) Va(tb) Pend(5px) D(n)--print">
     <input class="Pos(a) V(h)" type="checkbox"/>
     <svg class="Va(m)! H(16px) W(16px) Stk($plusGray)! Fill($plusGray)! Cur(p)" data-icon="checkbox-unchecked" height="16" style="fill:#000;stroke:#000;stroke-width:0;vertical-align:bottom" viewbox="0 0 24 24" width="16">
      <path d="M3 3h18v18H3V3zm19-2H2c-.553 0-1 .448-1 1v20c0 .552.447 1 1 1h20c.552 0 1-.448 1-1V2c0-.552-.448-1-1-1z">
      </path>
     </svg>
    </label>
    Symbol
    <div class="W(3px) Pend(5px) Pos(a) Start(100%)

The following code block shows how we can use a loop to iterate over all of the sectors, collecting the data stored in the table and using it to construct a pandas `DataFrame`.

In [19]:
data_list = []
for current_sector_loc in sector_urls:
    r = requests.get(current_sector_loc)
    soup = bs4.BeautifulSoup(r.content)

    for current_row in soup.find_all('tr'):
        row_list = []
        for current_data in current_row.find_all('td'):
            row_list.append(current_data.text)
        if row_list:
            row_list.append(current_sector_loc.split('/')[-1])
            row_list.append(datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S'))
            data_list.append(row_list)
    
columns = ['Symbol', 'Name', 'Price', 'Change', '%Change', 
           'Volume', 'Avg Volume (3 month)', 'Market Cap', 
           'PE Ratio', '52 Week Range', 'Sector', 'Scrape Time']

all_data = pd.DataFrame(data_list, columns = columns)
all_data.head()

,Symbol,Name,Price,Change,%Change,Volume,Avg Volume (3 month),Market Cap,PE Ratio,52 Week Range,Sector,Scrape Time
0,BHP,BHP Group Limited,78.29,-1.01,-1.27%,1.324M,6.064M,278.001B,11.77,,ms_basic_materials,04-04-2022 12:11:22
1,LIN,Linde plc,320.46,-2.27,-0.70%,"551,590",2.338M,162.712B,43.72,,ms_basic_materials,04-04-2022 12:11:22
2,RIO,Rio Tinto Group,81.02,-1.66,-2.01%,1.793M,5.096M,131.287B,6.26,,ms_basic_materials,04-04-2022 12:11:22
3,VALE,Vale S.A.,21.12,+0.30,+1.42%,15.887M,37.546M,102.346B,4.07,,ms_basic_materials,04-04-2022 12:11:22
4,CTA-PB,E. I. du Pont de Nemours and Company,95.10,+0.05,+0.05%,478,"1,627",82.528B,N/A,,ms_basic_materials,04-04-2022 12:11:22


## Wikipedia

Just to demonstrate a (slightly) more complex harvesting task, let's write some code to collect headlines from Wikipedia pages. As before, let's first check the `robots.txt` file, which is much more extensive.

In [ ]:
r = requests.get('https://en.wikipedia.org/robots.txt')
print(r.text)

The following code block creates a list of prohibited paths for a general web scraping application and displays the first 30 items in the list.

In [ ]:
prohibited_paths = []
current_user_agent = None
for line in r.text.split('\n'):
    if 'User-agent' in line:
        current_user_agent = line.strip('User-agent: ')
    if current_user_agent is not None:
        if current_user_agent.strip() == '*':
            if 'Disallow: ' in line:
                prohibited_paths.append(line.strip('Disallow: '))
                
prohibited_paths[:30]

If you browse through a few Wikipedia pages, you will notice that the links use a very consitent naming pattern. Suppose, we want to try the phrases in the `words_to_try` object.

In [ ]:
words_to_try = ['Stochastic_optimization', 
                'Robust_optimization',
                'Nonparametric_regression',
                'Data_warehouse',
                'tHis_wIll_nOt_wErk',
                'Integer_programming',
                'Cluster_analysis']

base_url = 'https://en.wikipedia.org/wiki/'

The following code attempts to access the pages defined by our `words_to_try` object and returns a list of the headlines along with the associated word.

In [ ]:
headlines = []

for current_word in words_to_try:
    current_url = base_url + current_word
    
    try:
        r = requests.get(current_url)
        soup = bs4.BeautifulSoup(r.content)

        for headline in soup.find_all('span', attrs = {'class':'mw-headline'}):
            if 'See also' in headline:
                break
            else:
                headlines.append([headline.text, current_word])
    except Exception as e:
        print(f'{type(e).__name__} on {current_url}')
    
    time.sleep(1)
    
for row in headlines:
    print(row)